In [1]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import json
import os
import joblib
from datetime import datetime
from typing import Optional
from dataclasses import dataclass


# 2. 서드파티 라이브러리 
import pandas as pd
import numpy as np

# 2-1. 시각화
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
import shap
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    f1_score, recall_score, precision_score, 
    roc_auc_score, average_precision_score
)
import optuna
from optuna.pruners import MedianPruner
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

In [2]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [3]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),무지고형분,MUN,305일유량,305일무지고형분,전산차비유지속성,전산차건유전유량,...,농장구분,정액코드분류,가격,가격미달,가격구분,분만간격,분만월령,공태일수_log,계절_sin,비유단계_sin
0,20249,20120709020022,2020-01-11,154,0,1,9714.0,805.0,50.5,16.1,...,1,NaN,1056.34,0,1,NaN,85,NaN,-2.449294e-16,1.000000e+00
1,20249,20120504020095,2020-01-11,52,0,1,NaN,NaN,68.8,22.6,...,1,NaN,1056.34,0,1,NaN,90,NaN,-2.449294e-16,1.000000e+00
2,20249,20111008020210,2020-01-11,115,0,1,11008.0,971.0,54.1,24.5,...,1,NaN,1060.34,0,1,NaN,95,NaN,-2.449294e-16,1.224647e-16
3,20249,20121014020049,2020-01-11,290,1,1,11318.0,1011.0,58.6,37.8,...,1,7HO,1062.31,0,1,NaN,77,5.4161,-2.449294e-16,-1.000000e+00
4,20249,20130812020216,2020-01-11,100,0,1,11324.0,957.0,68.1,20.5,...,1,NaN,1056.34,0,1,NaN,75,NaN,-2.449294e-16,1.224647e-16


In [4]:
df = milk

In [5]:
features = [
    # '누적착유일(연계)',
    '전산차비유지속성',
    '산차', 
    '농후사료급여량(연계)',
    # '공태일수',
    '공태일수_log',
    # '305일유량',
    '농장구분',
    # '분만간격',
    # '분만월령',
    '비유단계',
    # '비유단계_sin',
    '계절',
    # '계절_sin',
    '정액코드분류'
]

features = [
'전산차건유전유량',
'산차',
'농후사료급여량(연계)',
'공태일수',
'계절',
'농장구분',
'분만간격',
]

target = '가격미달'

## Train Test Split

In [ ]:
# ============================================
# 1. 데이터 로딩 (실제 데이터)
# ============================================
# df, features, target이 이미 정의되어 있다고 가정
# df: 전체 데이터프레임
# features: 피처 컬럼 리스트
# target: 타겟 컬럼명

# 시계열 기준 분할
split_date = '2021-08-01'
train = df[df['검정일자'] < split_date].copy()
test = df[df['검정일자'] >= split_date].copy()

X_train_full = train[features]
y_train_full = train[target]
X_test = test[features]
y_test = test[target]

print("="*60)
print("데이터 정보 (시계열 분할)")
print("="*60)
print(f"Train 기간: {train['검정일자'].min()} ~ {train['검정일자'].max()}")
print(f"Test 기간:  {test['검정일자'].min()} ~ {test['검정일자'].max()}")
print(f"\nTrain 샘플 수: {len(X_train_full):,}")
print(f"Test 샘플 수:  {len(X_test):,}")
print(f"Feature 수: {len(features)}")

# 클래스 분포 확인
print(f"\n클래스 분포 (Train):")
print(y_train_full.value_counts(normalize=True).sort_index())

print(f"\n클래스 분포 (Test):")
print(y_test.value_counts(normalize=True).sort_index())

# 결측치 확인
print(f"\n결측치 현황:")
missing_info = pd.DataFrame({
    'Feature': X_train_full.columns,
    '결측_개수': X_train_full.isnull().sum().values,
    '결측_비율(%)': (X_train_full.isnull().mean() * 100).values
}).sort_values('결측_비율(%)', ascending=False)
print(missing_info.head(10))
print(f"\n전체 결측 비율: {X_train_full.isnull().mean().mean()*100:.1f}%")

# 결측률 80% 이상 열 제거 (선택 사항)
missing_ratio = X_train_full.isnull().mean()
high_missing_cols = missing_ratio[missing_ratio > 0.8].index.tolist()
if high_missing_cols:
    print(f"\n⚠️  결측률 80% 이상 제거: {high_missing_cols}")
    X_train_full = X_train_full.drop(columns=high_missing_cols)
    X_test = X_test.drop(columns=high_missing_cols)
    features = [f for f in features if f not in high_missing_cols]

# ============================================
# 2. 범주형 변수 지정 및 전처리 🔥🔥🔥
# ============================================
print("\n" + "="*60)
print("범주형 변수 전처리")
print("="*60)

# 🔥 방법 1: 자동 감지 (object, category 타입)
categorical_features_auto = X_train_full.select_dtypes(include=['object', 'category']).columns.tolist()

# 🔥 방법 2: 수동 지정 (권장)
# categorical_features = ['축종', '성별', '등급', '농장코드']  # 실제 컬럼명으로 변경
categorical_features = []

if not categorical_features and categorical_features_auto:
    print(f"자동 감지된 범주형 변수: {categorical_features_auto}")
    use_auto = input("이 변수들을 범주형으로 사용하시겠습니까? (y/n): ")
    if use_auto.lower() == 'y':
        categorical_features = categorical_features_auto
    else:
        print("⚠️  범주형 변수를 수동으로 지정하세요:")
        print("   예: categorical_features = ['컬럼1', '컬럼2']")
        categorical_features = []

# 🔥🔥🔥 핵심: 범주형 변수 NaN 처리
if categorical_features:
    print(f"\n범주형 변수 처리 중: {categorical_features}")
    
    for col in categorical_features:
        # 1. NaN을 'Missing' 문자열로 변환
        X_train_full[col] = X_train_full[col].fillna('Missing')
        X_test[col] = X_test[col].fillna('Missing')
        
        # 2. string 타입으로 변환
        X_train_full[col] = X_train_full[col].astype(str)
        X_test[col] = X_test[col].astype(str)
        
        print(f"   ✅ {col}: NaN → 'Missing', dtype → str")
    
    # 범주형 변수 인덱스
    cat_features_indices = [features.index(c) for c in categorical_features if c in features]
    print(f"\n범주형 변수 인덱스: {cat_features_indices}")
else:
    cat_features_indices = None
    print("⚠️  범주형 변수 없음 (모두 수치형으로 처리)")

# ============================================
# 2. Validation 세트 분리 (시계열 고려)
# ============================================
# Train의 마지막 20%를 Validation으로 사용 (시계열 순서 유지)
valid_ratio = 0.2
valid_size = int(len(X_train_full) * valid_ratio)

X_train = X_train_full.iloc[:-valid_size].reset_index(drop=True)
y_train = y_train_full.iloc[:-valid_size].reset_index(drop=True)

X_valid = X_train_full.iloc[-valid_size:].reset_index(drop=True)
y_valid = y_train_full.iloc[-valid_size:].reset_index(drop=True)

print(f"\n시계열 기반 Train/Valid 분리:")
print(f"   Train: {len(X_train):,} 샘플 (80%)")
print(f"   Valid: {len(X_valid):,} 샘플 (20%, 시간상 최근)")
print(f"   Test:  {len(X_test):,} 샘플")

y_valid.value_counts()

데이터 정보 (시계열 분할)
Train 기간: 2020-01-01 00:00:00 ~ 2021-07-30 00:00:00
Test 기간:  2021-08-01 00:00:00 ~ 2022-02-04 00:00:00

Train 샘플 수: 31,743
Test 샘플 수:  8,521
Feature 수: 7

클래스 분포 (Train):
가격미달
0    0.894874
1    0.105126
Name: proportion, dtype: float64

클래스 분포 (Test):
가격미달
0    0.886046
1    0.113954
Name: proportion, dtype: float64

결측치 현황:
       Feature  결측_개수   결측_비율(%)
6         분만간격  20733  65.315188
3         공태일수  13326  41.980909
0     전산차건유전유량  12489  39.344107
1           산차      0   0.000000
2  농후사료급여량(연계)      0   0.000000
4           계절      0   0.000000
5         농장구분      0   0.000000

전체 결측 비율: 20.9%

범주형 변수 전처리
⚠️  범주형 변수 없음 (모두 수치형으로 처리)

시계열 기반 Train/Valid 분리:
   Train: 25,395 샘플 (80%)
   Valid: 6,348 샘플 (20%, 시간상 최근)
   Test:  8,521 샘플


가격미달
0    5494
1     854
Name: count, dtype: int64

In [ ]:
# ============================================
# 3. 범주형 변수 지정
# ============================================
# 🔥 실제 범주형 컬럼명으로 변경하세요
# 예: categorical_features = ['축종', '성별', '등급', '농장코드']
categorical_features = ['계절', '정액코드분류', '비유구분']  # 범주형 변수가 있으면 여기에 추가

# DataFrame 타입 기반 자동 감지 (선택 사항)
categorical_features_auto = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
if categorical_features_auto and not categorical_features:
    print(f"\n자동 감지된 범주형 변수: {categorical_features_auto}")
    use_auto = input("이 변수들을 범주형으로 사용하시겠습니까? (y/n): ")
    if use_auto.lower() == 'y':
        categorical_features = categorical_features_auto

if categorical_features:
    cat_features_indices = [features.index(c) for c in categorical_features if c in features]
    print(f"\n범주형 변수: {categorical_features}")
    print(f"범주형 인덱스: {cat_features_indices}")
else:
    cat_features_indices = None
    print("\n⚠️  범주형 변수 없음")

# ============================================
# 4. CatBoost Pool 생성
# ============================================
train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features_indices
)

valid_pool = Pool(
    data=X_valid,
    label=y_valid,
    cat_features=cat_features_indices
)

print("\n✅ CatBoost Pool 생성 완료")

# ============================================
# 5. Multi-Objective Optuna 함수 정의 (수정)
# ============================================
def objective_multi(trial):
    """
    Multi-Objective Optimization: Precision과 Recall 동시 최적화
    
    Returns:
    - (precision, recall): 둘 다 최대화
    """
    
    # 하이퍼파라미터 탐색 공간
    params = {
        'iterations': trial.suggest_int('iterations', 500, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1), 
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        
        # 🔥 subsample 사용 시 bootstrap_type 필수
        # 'bootstrap_type': 'Bernoulli',  # 🔥 추가: 'Bayesian' 대신 'Bernoulli' 사용
        # 'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        
        # 고정 파라미터
        # 'task_type': 'GPU',
        # 'devices': '0',
        'random_seed': 42,
        'verbose': False,
        'auto_class_weights': 'Balanced',
        'eval_metric': 'Precision',
        'early_stopping_rounds': 1000,
    }
    
    # 모델 학습
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=valid_pool, verbose=False, plot=False)
    
    # 검증 세트 예측
    y_pred = model.predict(X_valid)
    
    # Precision과 Recall 계산
    precision = precision_score(y_valid, y_pred, zero_division=0)
    recall = recall_score(y_valid, y_pred)
    
    return precision, recall

# ============================================
# 6. Optuna Multi-Objective Study 실행
# ============================================
print("\n" + "="*60)
print("🚀 Optuna Multi-Objective 최적화 시작")
print("   목표: Precision과 Recall 동시 최대화")
print("="*60)

# Multi-Objective Study 생성
study = optuna.create_study(
    directions=['maximize', 'maximize'],  # [precision, recall] 둘 다 최대화
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10),
    study_name='catboost_multi_objective'
)

# 최적화 실행
import time
start_time = time.time()

study.optimize(
    objective_multi,
    n_trials=300,  # 🔥 시간에 따라 조정 (50~200 권장)
    timeout=3600,  # 1시간 (초 단위)
    n_jobs=-1,  # GPU는 1, CPU는 -1 가능
    show_progress_bar=True
)

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("✅ Multi-Objective 최적화 완료")
print("="*60)
print(f"⏱️  소요 시간: {elapsed_time/60:.1f}분")
print(f"📊 총 시도 횟수: {len(study.trials)}")
print(f"🏆 Pareto 최적 해: {len(study.best_trials)}개")

[I 2025-11-09 17:03:33,785] A new study created in memory with name: catboost_multi_objective



범주형 변수: ['계절', '정액코드분류', '비유구분']
범주형 인덱스: [4]

✅ CatBoost Pool 생성 완료

🚀 Optuna Multi-Objective 최적화 시작
   목표: Precision과 Recall 동시 최대화


  0%|          | 0/300 [00:00<?, ?it/s]

[I 2025-11-09 17:03:48,987] Trial 11 finished with values: [0.2546230440967283, 0.41920374707259955] and parameters: {'iterations': 720, 'learning_rate': 0.062482066408885026, 'depth': 7, 'l2_leaf_reg': 6.524283243638025, 'random_strength': 9.489177363603682, 'bagging_temperature': 0.7101131121867668, 'min_data_in_leaf': 7, 'max_bin': 271}.
[I 2025-11-09 17:03:58,713] Trial 5 finished with values: [0.22226980728051393, 0.607728337236534] and parameters: {'iterations': 1077, 'learning_rate': 0.00847179886074828, 'depth': 7, 'l2_leaf_reg': 2.632802026503085, 'random_strength': 5.37304755870995, 'bagging_temperature': 0.917101159405729, 'min_data_in_leaf': 14, 'max_bin': 344}.
[I 2025-11-09 17:04:04,648] Trial 0 finished with values: [0.27353463587921845, 0.36065573770491804] and parameters: {'iterations': 1003, 'learning_rate': 0.037773938653391126, 'depth': 9, 'l2_leaf_reg': 3.0864558701494293, 'random_strength': 4.938598364154794, 'bagging_temperature': 0.4764587959998614, 'min_data_in

In [9]:
# ============================================
# 7. Pareto Front 분석
# ============================================
print("\n" + "="*60)
print("📈 Pareto 최적 해 분석")
print("="*60)

pareto_solutions = []
for i, trial in enumerate(study.best_trials):
    precision, recall = trial.values
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    
    pareto_solutions.append({
        'trial_number': trial.number,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'params': trial.params
    })
    
    print(f"\n#{i+1} (Trial {trial.number}):")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1:        {f1:.4f}")

# F1이 가장 높은 해 선택
pareto_df = pd.DataFrame(pareto_solutions)
best_f1_idx = pareto_df['f1'].idxmax()
best_solution = pareto_df.loc[best_f1_idx]

print("\n" + "="*60)
print("🎯 F1이 가장 높은 Pareto 해 선택")
print("="*60)
print(f"Trial Number: {best_solution['trial_number']}")
print(f"Precision: {best_solution['precision']:.4f}")
print(f"Recall:    {best_solution['recall']:.4f}")
print(f"F1:        {best_solution['f1']:.4f}")

best_params = best_solution['params']

print("\n📋 최적 하이퍼파라미터:")
for key, value in sorted(best_params.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")

# ============================================
# 8. Pareto Front 시각화
# ============================================
print("\n" + "="*60)
print("📊 Pareto Front 시각화")
print("="*60)

fig = go.Figure()

# 모든 trial
all_precisions = [t.values[0] for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
all_recalls = [t.values[1] for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

fig.add_trace(go.Scatter(
    x=all_recalls,
    y=all_precisions,
    mode='markers',
    name='모든 Trial',
    marker=dict(size=6, color='lightblue', opacity=0.5)
))

# Pareto 최적 해
pareto_recalls = pareto_df['recall']
pareto_precisions = pareto_df['precision']

fig.add_trace(go.Scatter(
    x=pareto_recalls,
    y=pareto_precisions,
    mode='markers+lines',
    name='Pareto Front',
    marker=dict(size=10, color='red', symbol='star'),
    line=dict(color='red', dash='dash')
))

# F1 최고점
fig.add_trace(go.Scatter(
    x=[best_solution['recall']],
    y=[best_solution['precision']],
    mode='markers',
    name=f'Best F1 ({best_solution["f1"]:.3f})',
    marker=dict(size=15, color='yellow', symbol='diamond', line=dict(width=2, color='black'))
))

fig.update_layout(
    title="Precision vs Recall (Pareto Front)",
    xaxis_title="Recall",
    yaxis_title="Precision",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    template="plotly_dark",
    showlegend=True
)

fig.show()

# ============================================
# 9. 최적 파라미터로 최종 모델 학습 (수정)
# ============================================
print("\n" + "="*60)
print("🔥 최적 파라미터로 최종 모델 학습")
print("="*60)

final_params = {
    **best_params,
    # 'bootstrap_type': 'Bernoulli',  # 🔥 명시적으로 추가 (혹시 best_params에 없을 경우 대비)
    # 'task_type': 'GPU',
    'devices': '0',
    'random_seed': 42,
    'verbose': 100,
    'auto_class_weights': 'Balanced',
    'eval_metric': 'Precision',
    'early_stopping_rounds': 50,
}

final_model = CatBoostClassifier(**final_params)
final_model.fit(train_pool, eval_set=valid_pool, plot=False)

print(f"\n✅ 최종 학습 완료 (iterations: {final_model.tree_count_})")

# ============================================
# 10. 임계값 후처리 (F1 최대화)
# ============================================
print("\n" + "="*60)
print("🎯 임계값 후처리 (F1 Score 최대화)")
print("="*60)

# 테스트 세트 확률 예측
y_prob = final_model.predict_proba(X_test)[:, 1]

# 임계값 범위 탐색
thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
best_f1 = 0
best_threshold = 0.5

for thr in thr_list:
    y_hat = (y_prob >= thr).astype(int)
    
    prec = precision_score(y_test, y_hat, zero_division=0)
    rec = recall_score(y_test, y_hat, zero_division=0)
    f1 = f1_score(y_test, y_hat, zero_division=0)
    
    rows.append([thr, prec, rec, f1])
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = thr

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

print(f"\n🏆 최적 임계값: {best_threshold}")
print(f"   → F1 Score: {best_f1:.4f}")

# 최적 임계값으로 예측
y_pred_optimal = (y_prob >= best_threshold).astype(int)

print(f"\n기본 임계값 (0.5) vs 최적 임계값 ({best_threshold}):")
y_pred_default = (y_prob >= 0.5).astype(int)

print(f"\n기본 (0.5):")
print(f"   Precision: {precision_score(y_test, y_pred_default, zero_division=0):.4f}")
print(f"   Recall:    {recall_score(y_test, y_pred_default):.4f}")
print(f"   F1:        {f1_score(y_test, y_pred_default, zero_division=0):.4f}")

print(f"\n최적 ({best_threshold}):")
print(f"   Precision: {precision_score(y_test, y_pred_optimal, zero_division=0):.4f}")
print(f"   Recall:    {recall_score(y_test, y_pred_optimal):.4f}")
print(f"   F1:        {f1_score(y_test, y_pred_optimal, zero_division=0):.4f}")

# ============================================
# 11. 임계값 변화 시각화 (Plotly)
# ============================================
print("\n" + "="*60)
print("📊 임계값 변화에 따른 메트릭 시각화")
print("="*60)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=thr_table["threshold"], 
    y=thr_table["precision"],
    mode="lines", 
    name="Precision",
    line=dict(dash="dash", color="cyan", width=2)
))

fig.add_trace(go.Scatter(
    x=thr_table["threshold"], 
    y=thr_table["recall"],
    mode="lines", 
    name="Recall",
    line=dict(color="magenta", width=2)
))

fig.add_trace(go.Scatter(
    x=thr_table["threshold"], 
    y=thr_table["f1"],
    mode="lines", 
    name="F1",
    line=dict(color="yellow", width=3)
))

# 최적 임계값 표시
fig.add_trace(go.Scatter(
    x=[best_threshold, best_threshold],
    y=[0, 1],
    mode="lines",
    name=f"최적 임계값 ({best_threshold})",
    line=dict(color="white", dash="dot", width=2)
))

# 최적 F1 점 표시
best_row = thr_table[thr_table['threshold'] == best_threshold].iloc[0]
fig.add_trace(go.Scatter(
    x=[best_threshold],
    y=[best_row['f1']],
    mode="markers",
    name=f"최고 F1 ({best_f1:.3f})",
    marker=dict(size=15, color="yellow", symbol="star", line=dict(width=2, color="black"))
))

fig.update_layout(
    title="임계값 변화에 따른 Precision/Recall/F1",
    xaxis_title="Threshold",
    yaxis_title="Score",
    yaxis=dict(range=[0, 1]),
    xaxis=dict(range=[0.1, 0.9]),
    template="plotly_dark",
    showlegend=True,
    hovermode='x unified'
)

fig.show()

# ============================================
# 12. 최종 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 최종 테스트 세트 평가 (최적 임계값 적용)")
print("="*60)

print(classification_report(
    y_test, y_pred_optimal,
    target_names=['0 (다수 클래스)', '1 (소수 클래스)']
))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred_optimal)
cm_df = pd.DataFrame(
    cm,
    index=['실제_0', '실제_1'],
    columns=['예측_0', '예측_1']
)
print(cm_df)

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=1 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (FP): {fp}개 → 다수를 소수로 오판")
print(f"   🚨 미탐 (FN): {fn}개 → 소수를 다수로 오판 (치명적!)")

if tp + fn > 0:
    recall = tp/(tp+fn)
    print(f"\n   📊 소수 클래스 Recall: {recall*100:.1f}%")
    print(f"      → 소수 {tp+fn}개 중 {tp}개 검출 ({fn}개 놓침)")

if tp + fp > 0:
    precision = tp/(tp+fp)
    print(f"   📊 소수 클래스 Precision: {precision*100:.1f}%")
    print(f"      → 소수 판정 {tp+fp}개 중 {tp}개 정답")

# 전체 메트릭
test_metrics = {
    'F1': f1_score(y_test, y_pred_optimal, zero_division=0),
    'Recall': recall_score(y_test, y_pred_optimal),
    'Precision': precision_score(y_test, y_pred_optimal, zero_division=0),
    'ROC-AUC': roc_auc_score(y_test, y_prob),
    'PR-AUC': average_precision_score(y_test, y_prob)
}

print("\n📊 전체 성능 메트릭:")
for metric, value in test_metrics.items():
    print(f"   {metric}: {value:.4f}")

# ============================================
# 13. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 20")
print("="*60)

feature_importance = final_model.get_feature_importance()
feature_names = X_train.columns

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print(importance_df.head(20).to_string(index=False))

# ============================================
# 14. PR Curve 시각화
# ============================================
print("\n" + "="*60)
print("📊 Precision-Recall Curve")
print("="*60)

from sklearn.metrics import precision_recall_curve

precisions_pr, recalls_pr, thresholds_pr = precision_recall_curve(y_test, y_prob)

fig_pr = go.Figure()

fig_pr.add_trace(go.Scatter(
    x=recalls_pr,
    y=precisions_pr,
    mode='lines',
    name=f'PR Curve (AUC={test_metrics["PR-AUC"]:.3f})',
    line=dict(color='cyan', width=3),
    fill='tozeroy',
    fillcolor='rgba(0, 255, 255, 0.2)'
))

# 최적 임계값 점 표시
optimal_idx = np.argmin(np.abs(thresholds_pr - best_threshold))
fig_pr.add_trace(go.Scatter(
    x=[recalls_pr[optimal_idx]],
    y=[precisions_pr[optimal_idx]],
    mode='markers',
    name=f'최적 임계값 ({best_threshold})',
    marker=dict(size=15, color='yellow', symbol='star', line=dict(width=2, color='black'))
))

fig_pr.update_layout(
    title=f"Precision-Recall Curve (PR-AUC: {test_metrics['PR-AUC']:.3f})",
    xaxis_title="Recall",
    yaxis_title="Precision",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    template="plotly_dark",
    showlegend=True
)

fig_pr.show()

# ============================================
# 15. 모델 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 저장")
print("="*60)

save_model = input("모델을 저장하시겠습니까? (y/n): ")
if save_model.lower() == 'y':
    from datetime import datetime
    from pathlib import Path
    import json
    import joblib
    
    MODEL_DIR = Path('.')
    MODEL_DIR.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # CatBoost 모델 저장
    model_filename = MODEL_DIR / f'catboost_model_multi_obj_{timestamp}.cbm'
    final_model.save_model(model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 파라미터 + 임계값 저장
    params_filename = MODEL_DIR / f'best_params_threshold_{timestamp}.json'
    params_info = {
        'best_params': best_params,
        'pareto_solutions': pareto_solutions,
        'best_f1_solution': {
            'precision': float(best_solution['precision']),
            'recall': float(best_solution['recall']),
            'f1': float(best_solution['f1']),
            'trial_number': int(best_solution['trial_number'])
        },
        'optimal_threshold': float(best_threshold),
        'optimal_threshold_f1': float(best_f1),
        'n_trials': len(study.trials),
        'n_pareto_solutions': len(study.best_trials),
        'split_date': split_date,
        'timestamp': timestamp,
        'categorical_features': categorical_features,
        'features': features,
        'target': target
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 + 임계값 저장: {params_filename}")
    
    # 테스트 성능 저장
    metrics_filename = MODEL_DIR / f'test_metrics_{timestamp}.json'
    metrics_info = {
        'test_metrics_default_threshold': {
            'threshold': 0.5,
            'f1': float(f1_score(y_test, y_pred_default, zero_division=0)),
            'precision': float(precision_score(y_test, y_pred_default, zero_division=0)),
            'recall': float(recall_score(y_test, y_pred_default))
        },
        'test_metrics_optimal_threshold': {
            'threshold': float(best_threshold),
            'f1': float(test_metrics['F1']),
            'precision': float(test_metrics['Precision']),
            'recall': float(test_metrics['Recall']),
            'roc_auc': float(test_metrics['ROC-AUC']),
            'pr_auc': float(test_metrics['PR-AUC'])
        },
        'confusion_matrix': {
            'tn': int(tn), 'fp': int(fp),
            'fn': int(fn), 'tp': int(tp)
        },
        'split_date': split_date,
        'train_period': f"{train['검정일자'].min()} ~ {train['검정일자'].max()}",
        'test_period': f"{test['검정일자'].min()} ~ {test['검정일자'].max()}",
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 임계값 테이블 CSV 저장
    thr_table_filename = MODEL_DIR / f'threshold_table_{timestamp}.csv'
    thr_table.to_csv(thr_table_filename, index=False)
    print(f"✅ 임계값 테이블 저장: {thr_table_filename}")
    
    # 변수 중요도 저장
    importance_filename = MODEL_DIR / f'feature_importance_{timestamp}.csv'
    importance_df.to_csv(importance_filename, index=False)
    print(f"✅ 변수 중요도 저장: {importance_filename}")
    
    print("\n📦 저장된 파일:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. 임계값 테이블: {thr_table_filename.name}")
    print(f"5. 변수 중요도: {importance_filename.name}")
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)

# ============================================
# 16. 최종 요약
# ============================================
print("\n" + "="*60)
print("📝 최종 요약")
print("="*60)
print(f"✅ 데이터 분할: {split_date} 기준")
print(f"   - Train: {train['검정일자'].min()} ~ {train['검정일자'].max()}")
print(f"   - Test:  {test['검정일자'].min()} ~ {test['검정일자'].max()}")
print(f"\n✅ Optuna Multi-Objective 최적화 완료")
print(f"   - Pareto 최적 해: {len(study.best_trials)}개")
print(f"   - 선택된 해 F1: {best_solution['f1']:.4f}")
print(f"\n✅ 임계값 후처리 완료")
print(f"   - 최적 임계값: {best_threshold}")
print(f"   - 최대 F1 Score: {best_f1:.4f}")
print(f"\n✅ 최종 테스트 성능")
print(f"   - F1: {test_metrics['F1']:.4f}")
print(f"   - Precision: {test_metrics['Precision']:.4f}")
print(f"   - Recall: {test_metrics['Recall']:.4f}")
print(f"   - PR-AUC: {test_metrics['PR-AUC']:.4f}")
print(f"\n⏱️  총 소요 시간: {elapsed_time/60:.1f}분")


📈 Pareto 최적 해 분석

#1 (Trial 15):
   Precision: 0.2357
   Recall:    0.6019
   F1:        0.3387

#2 (Trial 18):
   Precision: 0.3191
   Recall:    0.3314
   F1:        0.3251

#3 (Trial 26):
   Precision: 0.2635
   Recall:    0.4333
   F1:        0.3277

#4 (Trial 32):
   Precision: 0.3094
   Recall:    0.3536
   F1:        0.3301

#5 (Trial 33):
   Precision: 0.2443
   Recall:    0.5316
   F1:        0.3348

#6 (Trial 42):
   Precision: 0.3171
   Recall:    0.3361
   F1:        0.3263

#7 (Trial 58):
   Precision: 0.3134
   Recall:    0.3443
   F1:        0.3281

#8 (Trial 62):
   Precision: 0.2546
   Recall:    0.4344
   F1:        0.3211

#9 (Trial 73):
   Precision: 0.2197
   Recall:    0.6241
   F1:        0.3250

#10 (Trial 74):
   Precision: 0.2404
   Recall:    0.5351
   F1:        0.3318

#11 (Trial 84):
   Precision: 0.4249
   Recall:    0.1920
   F1:        0.2645

#12 (Trial 86):
   Precision: 0.3136
   Recall:    0.3419
   F1:        0.3272

#13 (Trial 94):
   Precision: 


🔥 최적 파라미터로 최종 모델 학습
0:	learn: 0.5767427	test: 0.6742482	best: 0.6742482 (0)	total: 6.51ms	remaining: 6.03s
100:	learn: 0.6705891	test: 0.7310684	best: 0.7310684 (100)	total: 475ms	remaining: 3.88s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7357489448
bestIteration = 112

Shrink model to first 113 iterations.

✅ 최종 학습 완료 (iterations: 113)

🎯 임계값 후처리 (F1 Score 최대화)

🏆 최적 임계값: 0.5
   → F1 Score: 0.2712

기본 임계값 (0.5) vs 최적 임계값 (0.5):

기본 (0.5):
   Precision: 0.1853
   Recall:    0.5057
   F1:        0.2712

최적 (0.5):
   Precision: 0.1853
   Recall:    0.5057
   F1:        0.2712

📊 임계값 변화에 따른 메트릭 시각화



🧪 최종 테스트 세트 평가 (최적 임계값 적용)
              precision    recall  f1-score   support

  0 (다수 클래스)       0.92      0.71      0.80      7550
  1 (소수 클래스)       0.19      0.51      0.27       971

    accuracy                           0.69      8521
   macro avg       0.55      0.61      0.54      8521
weighted avg       0.83      0.69      0.74      8521


혼동 행렬:
      예측_0  예측_1
실제_0  5391  2159
실제_1   480   491

💼 비즈니스 해석 (소수 클래스=1 기준):
   ✅ 정확한 분류: 5882개 (69.0%)
   ⚠️  오탐 (FP): 2159개 → 다수를 소수로 오판
   🚨 미탐 (FN): 480개 → 소수를 다수로 오판 (치명적!)

   📊 소수 클래스 Recall: 50.6%
      → 소수 971개 중 491개 검출 (480개 놓침)
   📊 소수 클래스 Precision: 18.5%
      → 소수 판정 2650개 중 491개 정답

📊 전체 성능 메트릭:
   F1: 0.2712
   Recall: 0.5057
   Precision: 0.1853
   ROC-AUC: 0.6355
   PR-AUC: 0.1984

📊 변수 중요도 Top 20
    feature  importance
         산차   23.015379
   전산차건유전유량   20.676232
       공태일수   17.989399
농후사료급여량(연계)   14.072976
       분만간격   12.835086
       농장구분    7.629460
         계절    3.781469

📊 Precision-Recall Curv


💾 모델 저장
✅ 모델 저장: catboost_model_multi_obj_20251109_172413.cbm
✅ 파라미터 + 임계값 저장: best_params_threshold_20251109_172413.json
✅ 테스트 성능 저장: test_metrics_20251109_172413.json
✅ 임계값 테이블 저장: threshold_table_20251109_172413.csv
✅ 변수 중요도 저장: feature_importance_20251109_172413.csv

📦 저장된 파일:
1. 모델: catboost_model_multi_obj_20251109_172413.cbm
2. 파라미터: best_params_threshold_20251109_172413.json
3. 성능: test_metrics_20251109_172413.json
4. 임계값 테이블: threshold_table_20251109_172413.csv
5. 변수 중요도: feature_importance_20251109_172413.csv

🎉 모든 작업 완료!

📝 최종 요약
✅ 데이터 분할: 2021-08-01 기준
   - Train: 2020-01-01 00:00:00 ~ 2021-07-30 00:00:00
   - Test:  2021-08-01 00:00:00 ~ 2022-02-04 00:00:00

✅ Optuna Multi-Objective 최적화 완료
   - Pareto 최적 해: 44개
   - 선택된 해 F1: 0.3448

✅ 임계값 후처리 완료
   - 최적 임계값: 0.5
   - 최대 F1 Score: 0.2712

✅ 최종 테스트 성능
   - F1: 0.2712
   - Precision: 0.1853
   - Recall: 0.5057
   - PR-AUC: 0.1984

⏱️  총 소요 시간: 19.0분


In [ ]:
models = [
    'best_lgbm_pr_auc_optimized_20251108_215407',
    'best_lgbm_pr_auc_optimized_20251108_222518',
    'best_lgbm_pr_auc_optimized_20251108_225509',
    'best_lgbm_pr_auc_optimized_20251108_233026',
    'best_lgbm_pr_auc_optimized_20251108_235658',
    'best_lgbm_pr_auc_optimized_20251109_002743',
    'best_lgbm_pr_auc_optimized_20251109_025011',
    'best_lgbm_pr_auc_optimized_20251109_033530',
    'best_lgbm_precision_optimized_20251109_041549'
]

In [ ]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_precision_optimized_20251109_041549.pkl')

In [ ]:
loaded_model

In [ ]:
# 테스트셋 성능 평가
y_pred = loaded_model.predict(X_test)
y_prob = loaded_model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.576).astype(int)

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)
print(f"ROC-AUC (Test Set): {roc_auc:.3f}")
print(f"PR-AUC (Test Set): {pr_auc:.4f}")

In [ ]:
thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
for thr in thr_list:
    y_hat = (y_prob >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0)
    ])

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["precision"],
                         mode="lines", name="Precision",
                         line=dict(dash="dash", color="cyan")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["recall"],
                         mode="lines", name="Recall",
                         line=dict(color="magenta")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["f1"],
                         mode="lines", name="F1",
                         line=dict(color="yellow")))

fig.update_layout(title="임곗값 변화에 따른 Precision/Recall/F1",
                  xaxis_title="Threshold",
                  yaxis_title="Score",
                  yaxis=dict(range=[0,1]),
                  template="plotly_dark")
fig.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import plotly.graph_objects as go

# 1) PR Curve 계산
prec, rec, thr = precision_recall_curve(y_test, y_prob)
ap = average_precision_score(y_test, y_prob)

# 2) 양성 비율 (baseline)
pos_rate = y_test.mean()

# 3) Plotly 시각화
fig_pr = go.Figure()

# PR 곡선
fig_pr.add_trace(go.Scatter(
    x=rec, y=prec,
    mode="lines", name=f"PR (AP={ap:.3f})",
    line=dict(color="magenta", width=3)
))

# 기준선 (양성 비율)
fig_pr.add_trace(go.Scatter(
    x=[0,1], y=[pos_rate, pos_rate],
    mode="lines", name=f"Baseline pos rate={pos_rate:.2f}",
    line=dict(dash="dash", color="grey")
))

fig_pr.update_layout(
    title="Precision-Recall Curve",
    xaxis_title="Recall",
    yaxis_title="Precision",
    template="plotly_dark",
    yaxis=dict(range=[0,1]),
    xaxis=dict(range=[0,1])
)

fig_pr.show()

In [ ]:
model = loaded_model.named_steps['model']

In [ ]:
import shap

# SHAP 적용
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot
shap.summary_plot(shap_values, X_test)


In [ ]:
# mean absolute shap value per feature
shap_importance = np.abs(shap_values).mean(axis=0)
print(shap_importance)
shap_df = pd.DataFrame({
    'feature': X_test.columns,
    'shap_importance': shap_importance
}).sort_values(by='shap_importance', ascending=False)

print(shap_df)